<a href="https://colab.research.google.com/github/Baby757/NB/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_2_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22cifar100_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras_flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [4]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [5]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [6]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255.0
X_val = X_val / 255.0

In [7]:
from keras.utils import np_utils

In [8]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train, 100)
y_val = np_utils.to_categorical(y_val, 100)
num_classes = y_val.shape[1]

In [9]:
# Импорт необходимых библиотек, которые мы можем использовать, а можем и не использовать. Всегда полезно импортировать.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,Input, AveragePooling2D, Activation,Conv2D, MaxPooling2D, BatchNormalization,Concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from matplotlib import pyplot

In [10]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [11]:
print('Shape of X_train is {}'.format(X_train.shape))
print('Shape of X_val is {}'.format(X_val.shape)) 
print('Shape of y_train is {}'.format(y_train.shape))
print('Shape of y_val is {}'.format(y_val.shape))

Shape of X_train is (50000, 32, 32, 3)
Shape of X_val is (10000, 32, 32, 3)
Shape of y_train is (50000, 100)
Shape of y_val is (10000, 100)


In [12]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, 2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [13]:
model1 = Sequential()
model1.add(Conv2D(16, (2, 2), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model1.add(Dropout(0.2))
model1.add(Conv2D(16, (2, 2), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(158, activation='relu', kernel_constraint=maxnorm(3)))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes, activation='softmax'))
# Копиляция model1
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.7, decay=decay, nesterov=False)
model1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 3, 32, 16)         2064      
                                                                 
 dropout (Dropout)           (None, 3, 32, 16)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 32, 16)         1040      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 16, 16)        0         
 )                                                               
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 158)               40606     
                                                      

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [14]:
# вычисление количества операций
flops = get_flops(model1, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


FLOPs: 0.7072e6


In [15]:
# Преобразуйте метки классов в one_hot формат
y_train = (y_train ==  np.arange(CLASSES)).astype(np.float32)
y_val = (y_val ==  np.arange(CLASSES)).astype(np.float32)

In [16]:
# вывод краткой информации о модели
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        208       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 16)       64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 16)        0         
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 100)               1638500   
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                        

In [17]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
     epochs=256
) 
# Окончательная оценка модели model1 
scores = model.evaluate(X_val, y_val, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/256
391/391 [==============================] - 59s 142ms/step - loss: 5.1910 - accuracy: 0.9875 - val_loss: 4.6746 - val_accuracy: 0.9640
Epoch 2/256
391/391 [==============================] - 57s 145ms/step - loss: 13.5145 - accuracy: 0.9890 - val_loss: 67.8215 - val_accuracy: 0.9887
Epoch 3/256
235/391 [=================>............] - ETA: 21s - loss: 13.9744 - accuracy: 0.9886